Import libraries

In [1]:
import pandas as pd

Load Data

In [3]:
df = pd.read_csv(r'data\pizza_sales.csv')

df.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1.0,1.0,hawaiian_m,1.0,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2.0,2.0,classic_dlx_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3.0,2.0,five_cheese_l,1.0,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4.0,2.0,ital_supr_l,1.0,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5.0,2.0,mexicana_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [7]:
df['order_date'].unique()

array(['1/1/2015', '2/1/2015', '3/1/2015', '4/1/2015', '5/1/2015',
       '6/1/2015', '7/1/2015', '8/1/2015', '9/1/2015', '10/1/2015',
       '11/1/2015', '12/1/2015', '13-01-2015', '14-01-2015', '15-01-2015',
       '16-01-2015', '17-01-2015', '18-01-2015', '19-01-2015',
       '20-01-2015', '21-01-2015', '22-01-2015', '23-01-2015',
       '24-01-2015', '25-01-2015', '26-01-2015', '27-01-2015',
       '28-01-2015', '29-01-2015', '30-01-2015', '31-01-2015', '1/2/2015',
       '2/2/2015', '3/2/2015', '4/2/2015', '5/2/2015', '6/2/2015',
       '7/2/2015', '8/2/2015', '9/2/2015', '10/2/2015', '11/2/2015',
       '12/2/2015', '13-02-2015', '14-02-2015', '15-02-2015',
       '16-02-2015', '17-02-2015', '18-02-2015', '19-02-2015',
       '20-02-2015', '21-02-2015', '22-02-2015', '23-02-2015',
       '24-02-2015', '25-02-2015', '26-02-2015', '27-02-2015',
       '28-02-2015', '1/3/2015', '2/3/2015', '3/3/2015', '4/3/2015',
       '5/3/2015', '6/3/2015', '7/3/2015', '8/3/2015', '9/3/2015',
   

Data Cleaning

In [ ]:
"""

def convert_date(date_str):
    try:
        # Пробуем преобразовать даты формата 'dd-mm-yyyy'
        return pd.to_datetime(date_str, format='%d-%m-%Y').strftime('%Y-%m-%d')
    except ValueError:
        # Если не получилось, пробуем формат 'm/d/yyyy'
        return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%Y-%m-%d')

df['order_date'] = df['order_date'].apply(convert_date)

"""

Create a database